In [1]:
import numpy as np
import math
import abc

### 1. Heston Model
$$\left.\left\{\begin{array}{l}dS_t=\mu S_tdt+\sqrt{v_t}S_tdB_t^S\\\\dv_t=\kappa(\theta-v_t)dt+\sigma_V\sqrt{v_t}dB_t^V\end{array}\right.\right.$$
According to Heston(1993), these can be transfered to a risk-neutral probability measure:
$$\left\{\begin{array}{l}dS_t=rS_tdt+\sqrt{v_t}S_td\widetilde{B}_t^S\\\\dv_t=\kappa^*(\theta^*-v_t)dt+\sigma_V\sqrt{v_t}d\widetilde{B}_t^V\end{array}\right.$$
where $\kappa^*=\kappa+\lambda,\quad\theta^*=\frac{\kappa\theta}{\kappa+\lambda}$, and $\lambda$ is the premium of volatility risk.

### 2. Variance Swap
The value of a variance swap at expiry :
$$V_T=(\sigma_R^2-K_{VAR})\times L$$
where $\sigma_R^2$ is the annualized realized variance, $K_{VAR}$ is the annualized delivery price, and $L$ is the notional amount of swap.

In risk-neutral world, the value of a variance swap at time t is the expected present value of the future payoff, $V_{t} = E_{t}^{Q}[e^{-r(T-t)}(\sigma_{R}^{2}-K_{var})L]$. This should be zero at the beginning of the contract since there is no cost to enter into a swap. Therefore, the fair variance delivery price can be easily defined as $K_{var}=E_0^Q[\sigma_R^2]$.

### 3. Fair strike price for the variance swap
The fair strike price for the variance swap is given as:
$$K_{var}=E_0^Q[\sigma_R^2]=\frac{e^{r\Delta t}}{T}[f(v_0)+\sum_{i=2}^Nf_i(v_0)]\times100^2$$
where r is interest rate, $\Delta t$ is defined as $\frac{T}{N}$ and N is the number of obersavation days.

Function $f(v_0)$ and $f_i(v_0)$ are defined as:
$$f(v)=e^{\tilde C(\Delta t)+\tilde D(\Delta t)v}+e^{-r\Delta t}-2$$
$$\begin{aligned}f_{i}(v_{0})&=e^{\tilde{C}(\Delta t)+\frac{c_{i}e^{-\kappa^{*}t_{i-1}}}{c_{i}-\tilde{D}(\Delta t)}\tilde{D}(\Delta t)v_{0}}(\frac{c_{i}}{c_{i}-\tilde{D}(\Delta t)})^{\frac{2\kappa^{*}\theta^{*}}{\sigma_{V}^{2}}}+e^{-r\Delta t}-2\end{aligned}$$
where $t_i = i\Delta t$, and $\kappa^{*}, \theta^{*}, \sigma_{V}$ are the parameters of Heston model under risk-neutral probability measure.

Function $\tilde{C}(\Delta t)$ and $\tilde{D}(\Delta t)$ are derived from the generalized Fourier transform method which is used to solving the PDE of payoff.  $\tilde{C}(\Delta t)$ and $\tilde{D}(\Delta t)$  are defined as:
$$\left\{\begin{array}{ll}\widetilde C(\tau)=r\tau+\frac{\kappa^*\theta^*}{\sigma_V^2}[(\widetilde a+\widetilde b)\tau-2\ln(\frac{1-\widetilde ge^{\widetilde b\tau}}{1-\widetilde g})]\\\\\widetilde D(\tau)=\frac{\widetilde a+\widetilde b}{\sigma_V^2}(\frac{1-e^{\widetilde b\tau}}{1-\widetilde ge^{\widetilde b\tau}})\\\\\widetilde a=\kappa^*-2\rho\sigma_V,\quad\widetilde b=\sqrt{\widetilde a^2-2\sigma_V^2},\quad\widetilde g=(\frac{\widetilde a}{\sigma_V})^2-1+(\frac{\widetilde a}{\sigma_V})\sqrt{(\frac{\widetilde a}{\sigma_V})^2-2}\\\end{array}\right.$$

In [2]:
class Variance_Swap(abc.ABC):
    def __init__(self, kappa_star, theta_star, rho, sigma_V, v0, r, N, T):
        self.kappa_star = kappa_star
        self.theta_star = theta_star
        self.rho = rho
        self.sigma_V = sigma_V
        self.v0 = v0
        self.r = r
        self.N = N
        self.T = T
        self.dt = T / N
        self.M = 200000
        self.S0 = 1
    def W_generation(self):
        W1 = np.random.standard_normal((self.N, self.M))
        W2 = np.random.standard_normal((self.N, self.M))
        return W1, W2

In [3]:
class Closed_Form_Solution(Variance_Swap):
    def C_D_calculation(self):
        tilde_a = self.kappa_star - 2 * self.rho * self.sigma_V
        tilde_b = np.sqrt(tilde_a ** 2 - 2 * self.sigma_V ** 2)
        tilde_g = (tilde_a / self.sigma_V) ** 2 - 1 + (tilde_a / self.sigma_V) * np.sqrt((tilde_a / self.sigma_V) ** 2 - 2)
        
        term1 = self.r * self.dt
        term2 = (self.kappa_star * self.theta_star) / (self.sigma_V ** 2)
        term3 = (tilde_a + tilde_b) * self.dt
        term4 = 2 * np.log((1 - tilde_g * np.exp(tilde_b * self.dt)) / (1 - tilde_g))
        
        C = term1 + term2 * (term3 - term4)

        D = ((tilde_a + tilde_b) / self.sigma_V ** 2) * ((1 - np.exp(tilde_b * self.dt)) / (1 - tilde_g * np.exp(tilde_b * self.dt)))
        return C, D
        
    def f(self):
        C, D = self.C_D_calculation()
        return np.exp(C + D * self.v0) + np.exp(-self.r * self.dt) - 2

    def sum_fi(self):
        C, D = self.C_D_calculation()
        sum = 0
        for i in range(2, self.N + 1):
            c_i = 2 * self.kappa_star / (self.sigma_V ** 2 * (1 - np.exp(-self.kappa_star * (i - 1) * self.dt)))
            term1 = np.exp(C + c_i * np.exp(-self.kappa_star * (i - 1) * self.dt) / (c_i - D) * D * self.v0)
            term2 = (c_i / (c_i - D)) ** (2 * self.kappa_star * self.theta_star / (self.sigma_V ** 2))
            sum += term1 * term2 + np.exp(-self.r * self.dt) - 2
        return sum
    
    def K_var(self):
        f_v0 = self.f()
        sum_fi_v0 = self.sum_fi()
        K_var = (np.exp(self.r * self.dt) / self.T) * (f_v0 + sum_fi_v0) * 100 ** 2
        return K_var


### Monte Carlo Simulation
By using the simple Euler-Maruyama discretization for the Heston model, we obtain:
$$\left\{\begin{array}{l}S_t=S_{t-1}+rS_{t-1}\Delta t+\sqrt{|v_{t-1}|}S_{t-1}\sqrt{\Delta t}W_t^1\\\\v_t=v_{t-1}+\kappa^*(\theta^*-v_{t-1})\Delta t+\sigma\sqrt{|v_{t-1}|}\sqrt{\Delta t}(\rho W_t^1+\sqrt{1-\rho^2}W_t^2)\end{array}\right.$$

Therefore, we calculate the realized variance by:
$$\sigma_R^2=\frac{AF}{N}\sum_{i=1}^N(\frac{S_{t_i}-S_{t_{i-1}}}{S_{t_{i-1}}})^2\times100^2$$
where N is the number of observations and AF is the annualized factor converting this expression to an annualized variance, $AF=\frac{1}{\Delta t}=\frac{N}{T}$.

In [4]:
class Numerical_Solution(Variance_Swap):

    def MC_simulation(self):
        W1, W2 = self.W_generation()
        sigma_path = np.zeros((self.N + 1, self.M))
        sigma_path[0] = self.v0
        for i in range(1, self.N + 1):
            sigma_path[i] = sigma_path[i-1] + self.kappa_star * (self.theta_star - sigma_path[i-1]) * self.dt
            + self.sigma_V * np.sqrt(abs(sigma_path[i-1]) * self.dt) * (self.rho * W1[i-1] + np.sqrt(1 - self.rho ** 2) * W2[i-1])
        
        S_path = np.zeros((self.N + 1, self.M))
        S_path[0] = self.S0
        for i in range(1, self.N + 1):
            S_path[i] = S_path[i-1] + self.r * S_path[i-1] * self.dt + np.sqrt(abs(sigma_path[i-1]) * self.dt) * S_path[i-1] * W1[i - 1]
        return S_path
    def Realized_vol(self):
        S_path = self.MC_simulation()
        S_path_pre = S_path[1:]
        S_path_lag = S_path[:-1]
        realized_vol = np.sum(((S_path_pre - S_path_lag)/S_path_lag)**2, axis = 0)
        return np.mean(realized_vol) * 100**2

In [5]:
for i in [4, 5, 6, 12, 26, 52, 252]:
    Discrete_Model = Closed_Form_Solution(kappa_star = 11.35, theta_star = 0.022, rho = -0.64, sigma_V = 0.618, v0 = 0.04, r = 0.1, N = i, T = 1).K_var()
    MC_simulation = Numerical_Solution(kappa_star = 11.35, theta_star = 0.022, rho = -0.64, sigma_V = 0.618, v0 = 0.04, r = 0.1, N = i, T = 1).Realized_vol()
    print('N = ', i, 'K_var = ', Discrete_Model, MC_simulation)

N =  4 K_var =  263.21430840556746 584.4665438352203
N =  5 K_var =  256.4864155589642 371.28959960831014
N =  6 K_var =  252.22828682744014 244.38435399930808
N =  12 K_var =  242.7208652168669 244.75712561947842
N =  26 K_var =  238.58347847937674 239.80173108525418
N =  52 K_var =  237.1097368799061 237.76449526393435
N =  252 K_var =  236.09696948601052 236.2766497550292
